# Import Libraries

In [ ]:
import pandas as pd
import numpy as np

#Libabries for GeoPlots
import geoviews as gv
import geoviews.feature as gf
import geopandas as gpd
from cartopy import crs as ccrs

#Libraries for Graphs
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import RangeToolLink

#Libraries for Bokeh Graphs
from bokeh.models.renderers import GlyphRenderer
from bokeh.models import Range1d, LinearAxis
from bokeh.models import HoverTool

#Libraries for Table
from bokeh.io import show, save, output_file
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn

#Library to Save the Plots
import panel as pn

hv.extension('bokeh')
gv.extension('bokeh')

# Import DataFrames and Initial Manipulation

In [ ]:
#Import and Rearrange DataFrames
##Gini Index Latest Values Database
GINI_data_latest = pd.read_csv('Gini_and_GNI_Index_Latest.csv')
GINI_data_latest = GINI_data_latest.drop(columns={'Country Name', 'GNI per capita PPP Rank'})
GINI_data_latest = GINI_data_latest.rename(columns={'Gini Index':'Gini_Index','GNI per capita PPP':'GNI_per_capita_PPP',
                                                    'Gini Index Rank': 'Rank', 'Country Code':'ISO3'})                                                    

##GeoPandas Database
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.rename(columns={'iso_a3':'ISO3'})
###Correcting GeoPandas Database
world.loc[world['name']=='Norway','ISO3'] = 'NOR'
world.loc[world['name']=='France','ISO3'] = 'FRA'
world.loc[world['name']=='Kosovo','ISO3'] = 'RKS'
world.loc[world['name']=='Kosovo','ISO3'] = 'RKS'

##Join DataFrames
df = world.merge(GINI_data_latest, on= 'ISO3', how='left')
df = df.rename(columns={'name':'Country'})
#Reordering columns
order = [0,1,2,3,4,6,7,8,5] # setting column's order
df = df[[df.columns[i] for i in order]]

## Creating the GeoMap Plot

In [ ]:
#Instatiating the Figures using Geoviews
##Gini Index Map
Gini_Index_map = gv.Polygons(df, vdims=['Gini_Index','Rank','Country'],label= 'Gini Index, 2017*'
           ).opts(projection=ccrs.Robinson(), width=800, height=400,tools=['hover'], colorbar=True, cmap='RdYlGn_r')
Gini_Index_map

In [ ]:
#Saving the Figures to HTML format
p = pn.panel(Gini_Index_map)
p.save('Gini_Index_Map.html')

# Graphs - Holoviews, based on Bokeh

## Creating Scatter Plot

In [ ]:
#Rearrange the DataFrame
df_scatter_full = pd.DataFrame(df)
df_scatter_full = df_scatter_full.drop(columns={'pop_est','ISO3','gdp_md_est','Rank','geometry'})
df_scatter_full = df_scatter_full.dropna()

In [ ]:
#Scatter Plot for Gini vs GNI
Gini_GNI_scatter = hv.Scatter(df_scatter_full, kdims = ['Gini_Index', 'GNI_per_capita_PPP'])

##Colors
explicit_mapping = {'Europe': 'lime', 'Asia': 'yellow', 'Oceania': 'red', 'North America': 'navy','South America': 'aqua',
                   'Africa':'saddlebrown'}

##Defining the Charts's Area

x_range = (df_scatter_full['Gini_Index'].min()-1,df_scatter_full['Gini_Index'].max()+1)
y_range = (df_scatter_full['GNI_per_capita_PPP'].min()-1,df_scatter_full['GNI_per_capita_PPP'].max()+1)
              
##Plot Joining all together
Gini_GNI_scatter = Gini_GNI_scatter.opts(opts.Scatter(tools=['hover'], height = 600, width=800, size = 10, 
                                                                xlim = x_range, ylim = y_range,
                                   color = 'continent', cmap=explicit_mapping, legend_position = 'top'))

# Create the Trendline
x = df_scatter_full['Gini_Index']
y = df_scatter_full['GNI_per_capita_PPP']
par = np.polyfit(x, y, 1, full=True)
slope=par[0][0]
intercept=par[0][1]
y_predicted = [slope*i + intercept  for i in x]

trendline = hv.Curve((x, y_predicted)).opts(opts.Curve(color='black', title="Inequality and Wealth by Country"))

full_scatter_GNI = Gini_GNI_scatter * trendline

#Save ScatterPlot to html file
p = pn.panel(full_scatter_GNI)
p.save('Inequality_vs_Wealth.html', embed = True)

## Creating a Graph with the 2 Variables with DropDown List

In [ ]:
dataframe = pd.read_csv('Gini_and_GNI.csv')

countries = dataframe.iloc[0,1:].unique()

#Need to Transform the Existing DataFrame into a Dictionary with several "DataFrames" by key (eg. Country) 
multi_df = dict()
for country in countries:
    #For each of the Variables will to the following procedure
    
    chosen_columns = []
    #This loop is to filter each variable's varlue in the big-dataframe and create a create a single Dataframe
    for column in dataframe.columns:
        if dataframe.iloc[0][column] == country:
            chosen_columns.append(dataframe[column])
    joint_table = pd.concat(chosen_columns, axis=1)

    #Some Small Changes to the obtained DataFrame
    joint_table['Code'] = dataframe['Code']

    country_table = joint_table[3:]
    country_table['Code'] = pd.to_datetime(country_table['Code'])
    country_table.columns = joint_table.iloc[1]
    country_table = country_table.rename(columns = {"Metric":"Date", "Gini Index": "Gini_Index",
                                                "GNI per capita PPP":"GNI_per_capita_PPP"})
    country_table = country_table.set_index('Date')
    country_table = country_table.apply(pd.to_numeric)
    
    #Joining the obtained Single Dataframes into the Dicitonary
    multi_df.update({country: country_table})

In [ ]:
#Defining the y-Axis Range
    #Because in the Dynamic Map below the Axis do not adjust in aconvenient way for both variables
y_max = []
y_min = []
for Index in multi_df.keys():
    y_min_ = min(multi_df[Index]['GNI_per_capita_PPP'].min(), multi_df[Index]['Gini_Index'].min())                    
    y_max_ = max(multi_df[Index]['GNI_per_capita_PPP'].max(), multi_df[Index]['Gini_Index'].max())

    y_min.append(y_min_)
    y_max.append(y_max_)

#Define the Final Range
y_range = (round(min(y_min)), round(max(y_max)))

In [ ]:
#Define the Function to be used in the Dynamic Map
def load_countries(Country):
    #Defining the Hovering Tooltips
    hover_1 = HoverTool(tooltips=[("Date", "@x{%F}"), ("GNI", "@y")], formatters={'x': 'datetime'})   
    hover_2 = HoverTool(tooltips=[("Date", "@x{%F}"), ("Gini_Index", "@y")], formatters={'x': 'datetime'})   
    
    #Instatiating the Curve Objects for Plotting
    c_GNI = hv.Curve((multi_df[Country].index,  multi_df[Country]['GNI_per_capita_PPP']
                      ), label='GNI_per_capita_PPP').opts(tools=[hover_1], color='red', xlabel='Date', ylim= y_range,
                                                          ylabel = 'Index/ Thousands of $')

    c_Gini = hv.Curve((multi_df[Country].index, multi_df[Country]['Gini_Index']
                     ),label = 'Gini_Index').opts(tools=[hover_2], color='blue', ylim= y_range,
                                                  ylabel = 'Index/ Thousands of $')
    #Join the Curve Objects and create the Complete Graph
    full_graph = hv.Overlay(c_GNI * c_Gini).opts(width=700, height = 400, legend_position='top_right')
    full_graph
    
    return full_graph

#Define what are the possible entries in the DropDown List
countries_name = countries

#Instantiate the DynamicMap object
dmap = hv.DynamicMap(load_countries, kdims='Country').redim.values(Country=countries_name)

#Saving the Graph to HTML format
p = pn.panel(dmap)
p.save('GNI_vs_Gini_Many_Countries.html', embed = True)

# Save the DataFrame to html format

In [ ]:
#DataFrame for Table
df_table = pd.read_csv('Gini_and_GNI_Index_Latest.csv')
df_table = df_table.drop(columns={'Country Code'})
df_table = df_table.rename(columns={'Country Name':'Country'}) 

In [ ]:
df_columns = df_table.columns

#Define the Source and the Columns of the DataFrame
source = ColumnDataSource(df_table)

columns = [TableColumn(field=Ci, title=Ci, width=20) for Ci in df_columns]

#Create the DataFrame
data_table = DataTable(source=source, columns=columns, width=700, height=400, selectable = True, index_position = None)

#Save the DataFrame
output_file('Data_Table.html')
save(data_table)

# Extra: Individual map with 2 axis 

In [ ]:
#Function to Create a Secondary Axis
def apply_formatter(plot, element):
    p = plot.state

    # Create secondary range and axis
    p.extra_y_ranges = {"twiny": Range1d(start=country_table['Gini_Index'].min()-1, end=country_table['Gini_Index'].max()+1)}

    p.add_layout(LinearAxis(axis_label = 'Gini_Index', y_range_name="twiny"), 'right')

    # Set glyph y_range_name to the one that was just created
    glyph = p.select(dict(type=GlyphRenderer))[0]
    glyph.y_range_name = 'twiny'

#Defining the Hovering Tooltips
hover_1 = HoverTool(tooltips=[("Date", "@x{%F}"), ("GNI_per_capita_PPP", "@y")], formatters={'x': 'datetime'})   
hover_2 = HoverTool(tooltips=[("Date", "@x{%F}"), ("Gini_Index", "@y")], formatters={'x': 'datetime'})   

#Instatiating the Curve Objects for Plotting
c_GNI = hv.Curve((country_table.index,  country_table['GNI_per_capita_PPP']
                  ), label='GNI_per_capita_PPP').opts(tools=[hover_1], color='red', xlabel='Date',ylabel = 'GNI_per_capita_PPP')

c_Gini = hv.Curve((country_table.index, country_table['Gini_Index']
                 ),label = 'Gini_Index').opts(tools=[hover_2], color='blue', hooks=[apply_formatter])

#Join the Curve Objects and create the Complete Graph
full_graph = (c_GNI * c_Gini).opts(width=800, height = 400, legend_position='top_left', title='Portugal')

#Saving the Graph to HTML format
p = pn.panel(full_graph)
p.save('Portugal_Gini_GNI.html', embed = True)